In [20]:
import numpy as np
import matplotlib.pyplot as plt
from xml.dom import minidom
from collections import defaultdict
import xml.etree.ElementTree as ET
import math
import umap
import os
import re
import PIL
import pickle
from pprint import pprint
from svg.path import parse_path
from xml.dom import minidom
from collections import Counter
from tqdm import tqdm
from bs4 import BeautifulSoup

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

np.random.seed(42)


In [15]:
#svg_dom = parse("./svg_datasets/chartblocks/charts/0a0a3a73-56e6-4a5d-9a65-45fa77288444/svg.txt")
# Reading the data inside the xml
# file to a variable under the name 
# data
with open("./svg_datasets/chartblocks/charts/0a0a3a73-56e6-4a5d-9a65-45fa77288444/svg.txt", 'r') as f:
#with open("./test.svg", 'r') as f:
    data = f.read()
 
# Passing the stored data inside
# the beautifulsoup parser, storing
# the returned object 
Bs_data = BeautifulSoup(data, "xml")

for tag in Bs_data.find_all():
    print(tag.attrs)

{'version': '1.1', 'style': 'font-family: Arial;', 'width': '711', 'height': '399', 'viewBox': '0 0 711 399', 'xmlns': 'http://www.w3.org/2000/svg'}
{}
{}
{'class': 'scaler', 'transform': 'scale(0.88875)'}
{'width': '800', 'height': '450', 'fill': 'rgb(255, 255, 255)', 'fill-opacity': '1'}
{'class': 'canvas', 'transform': 'translate(96,72)'}
{'class': 'grid'}
{'class': 'axis-grid x', 'transform': 'translate(0,303)'}
{'class': 'axis-grid y', 'transform': 'translate(1,0)'}
{'class': 'axis-grid y2', 'transform': 'translate(486.5,0)'}
{'class': 'axis-grid x2', 'transform': 'translate(0,1)'}
{'class': 'trigger-layer', 'height': '379', 'width': '724', 'opacity': '0'}
{'class': 'chart'}
{'class': 'series s0'}
{'class': 'bar'}
{'fill': 'rgb(120, 140, 139)', 'opacity': '1', 'x': '32.3125', 'width': '17.321428571428573', 'y': '301.8627272727273', 'height': '0.1372727272727161', 'clip-path': 'url(#chart-clipPath-exact)'}
{'class': 'bar'}
{'fill': 'rgb(120, 140, 139)', 'opacity': '1', 'x': '183.87

In [15]:
id_to_chart_type = {
    -1: "other",
    1: "line",
    2: "scatter",
    4: "bar",
    7: "heat_map",
    9: "box-plot",
    10: "bubble",
    13: "sankey",
    14: "chord",
    15: "radial",
    16: "area",
    18: "donut",
    19: "choropleth",
    22: "treemap",
    29: "pie",
    31: "stream_graph",
    33: "hexabin",
    35: "graph",
    37: "parallel_coordinates",
    38: "sunburst",
    39: "waffle",
    40: "voronoi",
    41: "word_cloud",
    60: "contour",
    61: "filled-line",
    62: "scattergeo"
}
chart_type_to_id = {v: k for k, v in id_to_chart_type.items()}

In [7]:
def extract_svg_features(svg_path, return_dict=False):
    with open(svg_path, 'r') as f:
        svg_text = f.read()

    svg_data = BeautifulSoup(svg_text, "xml")

    svg_features = {
        "num_path_h_lines": 0, # H or h
        "num_path_v_lines": 0, # V or v
        "num_path_q_points": 0, # Q or q
        "num_path_a_points": 0 # A or a
    }
    svg_height = 0
    svg_width = 0
    svg_diagonal_len = 0
    tag_to_count = {"rect": 0, "line": 0, "path": 0, "circle": 0, "text": 0}
    tag_class_to_count = {"line": 0, "surface": 0, "annotation": 0, "slice": 0, "circle": 0}
    svg_positions = []
    svg_x_positions = []
    svg_y_positions = []
    #svg_line_lens = []
    svg_path_num_pts = []
    svg_circle_radii = []
    svg_rect_widths = []
    svg_rect_heights = []
    svg_unique_classes = set()
    svg_unique_fill_colors = set()
    svg_unique_stroke_colors = set()
    svg_stroke_widths = []
    svg_font_sizes = []

    for child in svg_data.find_all():
        tag_type = child.name
        cur_attrib = child.attrs
        #print(tag_type, cur_attrib)
        if tag_type in tag_to_count:
            tag_to_count[tag_type] += 1

        if "class" in cur_attrib:
            svg_unique_classes.add(cur_attrib["class"])
            if cur_attrib["class"] in tag_class_to_count:
                tag_class_to_count[cur_attrib["class"]] += 1
        if "style" in cur_attrib:
            fill = re.search(r"fill: (.*?[\da-zA-Z(),\s#]+)", cur_attrib["style"])
            stroke = re.search(r"stroke: (.*?[\da-zA-Z(),\s#]+)", cur_attrib["style"])
            stroke_width = re.search(r"stroke-width: (.*?[\da-zA-Z(),\s#]+)", cur_attrib["style"])
            if fill and "none" not in fill.group(1):
                svg_unique_fill_colors.add(fill.group(1))
            if stroke and "none" not in stroke.group(1):
                svg_unique_stroke_colors.add(stroke.group(1))
            if stroke_width and "none" not in stroke_width.group(1):
                svg_stroke_widths.append(float(re.sub(r'[a-zA-Z]', "", stroke_width.group(1))))
            if tag_type == "text":
                font_size = re.search(r"font-size: (.*?[\da-zA-Z(),\s#]+)", cur_attrib["style"])
                if font_size and "none" not in font_size.group(1):
                    svg_font_sizes.append(float(re.sub(r'[a-zA-Z]', "", font_size.group(1))))

        if "x" in cur_attrib and "y" in cur_attrib:
            svg_positions.append((float(cur_attrib["x"]), float(cur_attrib["y"])))
            svg_x_positions.append(float(cur_attrib["x"]))
            svg_y_positions.append(float(cur_attrib["y"]))

        if tag_type == "svg":
            svg_width = float(re.sub(r'[a-zA-Z]', "", cur_attrib["width"]))
            svg_height = float(re.sub(r'[a-zA-Z]', "", cur_attrib["height"]))
            svg_diagonal_len = math.sqrt(svg_width**2 + svg_height**2)
        elif tag_type == "rect":
            svg_rect_widths.append(float(cur_attrib["width"]))
            svg_rect_heights.append(float(cur_attrib["height"]))
        elif tag_type == "circle":
            svg_circle_radii.append(float(cur_attrib["r"]))
        elif tag_type == "path" and "d" in cur_attrib:
            d_attrib = cur_attrib["d"].lower()
            svg_features["num_path_h_lines"] += d_attrib.count("h")
            svg_features["num_path_v_lines"] += d_attrib.count("v")
            svg_features["num_path_a_points"] += d_attrib.count("a")
            svg_features["num_path_q_points"] += d_attrib.count("q")
            svg_path_num_pts.append(len(re.findall(r"[a-z]", d_attrib)))

    svg_features["rect_count"] = tag_to_count["rect"]
    svg_features["line_count"] = tag_to_count["line"]
    svg_features["path_count"] = tag_to_count["path"]
    svg_features["circle_count"] = tag_to_count["circle"]
    svg_features["text_count"] = tag_to_count["text"]
    #svg_features["line_class_count"] = tag_class_to_count["line"]
    #svg_features["surface_class_count"] = tag_class_to_count["surface"]
    #svg_features["annotation_class_count"] = tag_class_to_count["annotation"]
    #svg_features["circle_class_count"] = tag_class_to_count["circle"]
    #svg_features["slice_class_count"] = tag_class_to_count["slice"]
    svg_features["num_unique_classes"] = len(svg_unique_classes)
    svg_features["num_unique_fill_colors"] = len(svg_unique_fill_colors)
    svg_features["num_unique_stroke_colors"] = len(svg_unique_stroke_colors)
    svg_features["num_unique_font_sizes"] = len(set(svg_font_sizes))

    if len(svg_font_sizes) == 0:
        svg_features["font_size_std"] = 0
        svg_features["font_size_min"] = 0
        svg_features["font_size_max"] = 0
    else:
        svg_features["font_size_std"] = np.std(svg_font_sizes)
        svg_features["font_size_min"] = np.min(svg_font_sizes)
        svg_features["font_size_max"] = np.max(svg_font_sizes)

    if len(svg_stroke_widths) == 0:
        svg_features["stroke_width_min"] = 0
        svg_features["stroke_width_max"] = 0
    else:
        svg_features["stroke_width_min"] = np.min(svg_stroke_widths)
        svg_features["stroke_width_max"] = np.max(svg_stroke_widths)

    positions_counter = Counter(svg_positions)
    svg_features["avg_duplicate_positions"] = np.mean([v for k, v in positions_counter.items()])
    svg_features["total_unique_positions"] = len(positions_counter)
    norm_x_pos = np.array(svg_x_positions) / svg_width
    norm_y_pos = np.array(svg_y_positions) / svg_height
    svg_features["x_pos_mean"] = np.mean(norm_x_pos)
    svg_features["x_pos_std"] = np.std(norm_x_pos)
    svg_features["x_pos_max"] = np.max(norm_x_pos)
    svg_features["x_pos_min"] = np.min(norm_x_pos)
    svg_features["y_pos_mean"] = np.mean(norm_y_pos)
    svg_features["y_pos_std"] = np.std(norm_y_pos)
    svg_features["y_pos_max"] = np.max(norm_y_pos)
    svg_features["y_pos_min"] = np.min(norm_y_pos)

    #if len(svg_circle_radii) == 0:
    #    svg_features["circle_radii_max_duplicates"] = 0
    #    svg_features["circle_radii_std"] = 0
    #    svg_features["circle_radii_max"] = 0
    #    svg_features["circle_radii_min"] = 0
    #else:
    #    circle_radii_counter = Counter(svg_circle_radii)
    #    norm_radii = np.aray(svg_circle_radii) / svg_diagonal_len
    #    svg_features["circle_radii_max_duplicates"] = np.max(list(circle_radii_counter.values()))
    #    svg_features["circle_radii_std"] = np.std(norm_radii)
    #    svg_features["circle_radii_max"] = np.max(norm_radii)
    #    svg_features["circle_radii_min"] = np.min(norm_radii)

    if len(svg_rect_widths) == 0:
        svg_features["rect_width_max_duplicates"] = 0
        svg_features["rect_width_std"] = 0
        svg_features["rect_width_max"] = 0
        svg_features["rect_width_min"] = 0
        svg_features["rect_height_max_duplicates"] = 0
        svg_features["rect_height_std"] = 0
        svg_features["rect_height_max"] = 0
        svg_features["rect_height_min"] = 0
    else:
        rect_width_counter = Counter(svg_rect_widths)
        norm_rect_widths = np.array(svg_rect_widths) / svg_width
        svg_features["rect_width_max_duplicates"] = np.max(list(rect_width_counter.values()))
        svg_features["rect_width_std"] = np.std(norm_rect_widths)
        svg_features["rect_width_max"] = np.max(norm_rect_widths)
        svg_features["rect_width_min"] = np.min(norm_rect_widths)

        rect_height_counter = Counter(svg_rect_heights)
        norm_rect_heights = np.array(svg_rect_heights) / svg_height
        svg_features["rect_height_max_duplicates"] = np.max(list(rect_height_counter.values()))
        svg_features["rect_height_std"] = np.std(norm_rect_heights)
        svg_features["rect_height_max"] = np.max(norm_rect_heights)
        svg_features["rect_height_min"] = np.min(norm_rect_heights)

    if len(svg_path_num_pts) == 0:
        svg_features["num_path_pts_std"] = 0
        svg_features["num_path_pts_mean"] = 0
        svg_features["num_path_pts_max"] = 0
        svg_features["num_path_pts_min"] = 0
    else:
        svg_features["num_path_pts_std"] = np.std(svg_path_num_pts)
        svg_features["num_path_pts_mean"] = np.mean(svg_path_num_pts)
        svg_features["num_path_pts_max"] = np.max(svg_path_num_pts)
        svg_features["num_path_pts_min"] = np.min(svg_path_num_pts)

    if return_dict:
        return svg_features
    else:
        svg_feature_arr = []
        svg_feature_names = []
        for feature_name, feature_val in svg_features.items():
            svg_feature_arr.append(feature_val)
            svg_feature_names.append(feature_name)
        return svg_feature_arr, svg_feature_names


In [120]:
pprint(extract_svg_features("./svg_datasets/plotly_export/charts/312/svg.txt", return_dict=True))

{'avg_duplicate_positions': 2.272727272727273,
 'circle_count': 0,
 'font_size_max': 14.0,
 'font_size_min': 12.0,
 'font_size_std': 0.6614378277661477,
 'line_count': 0,
 'num_path_a_points': 1834,
 'num_path_h_lines': 14,
 'num_path_pts_max': 739,
 'num_path_pts_mean': 5.787488500459982,
 'num_path_pts_min': 2,
 'num_path_pts_std': 26.15019089331409,
 'num_path_q_points': 0,
 'num_path_v_lines': 11,
 'num_unique_classes': 52,
 'num_unique_fill_colors': 6,
 'num_unique_font_sizes': 2,
 'num_unique_stroke_colors': 4,
 'path_count': 1087,
 'rect_count': 11,
 'rect_height_max': 1.0,
 'rect_height_max_duplicates': 4,
 'rect_height_min': 0.0,
 'rect_height_std': 0.3742467476133541,
 'rect_width_max': 1.0,
 'rect_width_max_duplicates': 4,
 'rect_width_min': 0.0,
 'rect_width_std': 0.3603767222763508,
 'stroke_width_max': 2.0,
 'stroke_width_min': 0.0,
 'text_count': 16,
 'total_unique_positions': 11,
 'x_pos_max': 0.99,
 'x_pos_mean': 0.07697142857142858,
 'x_pos_min': -0.01,
 'x_pos_std': 

In [121]:
data_dirs = ["./svg_datasets/plotly_export", "./svg_datasets/chartblocks", "./svg_datasets/d3_clean", "./svg_datasets/graphiq_clean"]
#["./svg_datasets/plotly_export", "./svg_datasets/chartblocks", "./svg_datasets/d3_clean", "./svg_datasets/graphiq_clean"]
X = []
y = []
for charts_dir in data_dirs:
    labels_path = f"{charts_dir}/urls.txt"
    id_to_label = {}

    with open(labels_path) as labels_file:
        for label_line in labels_file.readlines():
            label_split = label_line.split(" ")
            sample_id = label_split[0]
            sample_label = label_split[2].split(",")[0]
            if "plotly" in charts_dir:
                id_to_label[sample_id] = chart_type_to_id[sample_label.replace("\n", "")]
            else:
                id_to_label[sample_id] = int(sample_label.replace("\n", ""))

    err_count = 0
    processed_count = 0
    for chart_id, chart_label in tqdm(id_to_label.items()):
        svg_path = f"{charts_dir}/charts/{chart_id}/svg.txt"
        #print(svg_path)
        try:
            svg_feature_arr, svg_feature_names = extract_svg_features(svg_path)
            X.append(svg_feature_arr)
            y.append(chart_label)
            #processed_count += 1
            #if processed_count > 1000:
            #    break
        except Exception as err:
            err_count += 1

    print(f"Percentage of docs with parsing errors: {err_count / len(id_to_label)}")



  0%|          | 0/15232 [00:00<?, ?it/s]

  7%|▋         | 1015/15232 [00:17<06:16, 37.77it/s]/Users/kainylund/opt/anaconda3/envs/arkenv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/kainylund/opt/anaconda3/envs/arkenv/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/kainylund/opt/anaconda3/envs/arkenv/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/kainylund/opt/anaconda3/envs/arkenv/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/kainylund/opt/anaconda3/envs/arkenv/lib/python3.10/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid v

Percentage of docs with parsing errors: 0.019170168067226892


 48%|████▊     | 10778/22557 [00:54<00:54, 215.39it/s]/var/folders/76/70fs0djn7lb2y98w5_hxbtx00000gn/T/ipykernel_71140/3330749255.py:115: RuntimeWarning: divide by zero encountered in divide
  norm_y_pos = np.array(svg_y_positions) / svg_height
/var/folders/76/70fs0djn7lb2y98w5_hxbtx00000gn/T/ipykernel_71140/3330749255.py:115: RuntimeWarning: invalid value encountered in divide
  norm_y_pos = np.array(svg_y_positions) / svg_height
/Users/kainylund/opt/anaconda3/envs/arkenv/lib/python3.10/site-packages/numpy/core/_methods.py:180: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/kainylund/opt/anaconda3/envs/arkenv/lib/python3.10/site-packages/numpy/core/_methods.py:213: RuntimeWarning: invalid value encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)
/var/folders/76/70fs0djn7lb2y98w5_hxbtx00000gn/T/ipykernel_71140/3330749255.py:156: RuntimeWarning: divide by zero encountered in div

Percentage of docs with parsing errors: 0.0


 99%|█████████▉| 1423/1440 [00:29<00:00, 52.58it/s]/var/folders/76/70fs0djn7lb2y98w5_hxbtx00000gn/T/ipykernel_71140/3330749255.py:114: RuntimeWarning: invalid value encountered in divide
  norm_x_pos = np.array(svg_x_positions) / svg_width
/Users/kainylund/opt/anaconda3/envs/arkenv/lib/python3.10/site-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
100%|██████████| 1440/1440 [00:30<00:00, 47.57it/s]


Percentage of docs with parsing errors: 0.5625


100%|██████████| 2733/2733 [00:11<00:00, 241.92it/s]

Percentage of docs with parsing errors: 0.0761068422978412


In [1]:
dev_pct = 0.1
test_pct = 0.2
shuffled_idxs = np.random.choice(len(y), size=len(y), replace=False)

num_dev_elements = int(len(y) * dev_pct)
num_test_elements = int(len(y) * test_pct)
num_train_elements = int(len(y) * (1.0 - dev_pct - test_pct))

print(num_dev_elements, num_test_elements, num_train_elements)

train_idxs = shuffled_idxs[:num_train_elements]
dev_idxs = shuffled_idxs[num_train_elements:(num_train_elements + num_dev_elements)]
test_idxs = shuffled_idxs[(num_train_elements + num_dev_elements + 1):]

X = np.array(X)
X[X > 1e300] = 0.0
X[X < -1e300] = 0.0
X[np.isnan(X)] = 0.0
X_train = X[train_idxs]
X_dev = X[dev_idxs]
X_test = X[test_idxs]

y = np.array(y)
y_train = y[train_idxs]
y_dev = y[dev_idxs]
y_test = y[test_idxs]

np.save("./svg_datasets/svgs_X_train", X_train)
np.save("./svg_datasets/svgs_X_dev", X_dev)
np.save("./svg_datasets/svgs_X_test", X_test)
np.save("./svg_datasets/svgs_y_train", y_train)
np.save("./svg_datasets/svgs_y_dev", y_dev)
np.save("./svg_datasets/svgs_y_test", y_test)

NameError: name 'np' is not defined

In [3]:
X_train = np.load("./svg_datasets/svgs_X_train.npy")
X_dev = np.load("./svg_datasets/svgs_X_dev.npy")
y_train = np.load("./svg_datasets/svgs_y_train.npy")
y_dev = np.load("./svg_datasets/svgs_y_dev.npy")

In [4]:
print(np.max(X_train), np.min(X_train))
print(np.max(X_dev), np.min(X_dev))

X_dev[X_dev < -1e300] = 0.0

print(np.max(X_dev), np.min(X_dev))
np.save("./svg_datasets/svgs_X_dev.npy", X_dev)

196011.0 -280.253164556962
117576.0 -inf
117576.0 -731.402205882353


In [13]:
cls_names = [
    "nearest_neighbors",
    "linear_SVM",
    "RBF_SVM",
    "decision_tree",
    "random_forest",
    "neural_net",
    "adaBoost",
    "naive_bayes"
]

for name in cls_names:
    with open(f"./classifiers/{name}_svg_classifier.pkl", 'rb') as cls_file:
        svg_cls = pickle.load(cls_file)

    line_features, _ = extract_svg_features("./test_bikes.svg", return_dict=False)
    bar_features, _ = extract_svg_features("./test_bars.svg", return_dict=False)
    scatter_features, _ = extract_svg_features("./test_scatter.svg", return_dict=False)
    heatmap_features, _ = extract_svg_features("./test_heatmap.svg", return_dict=False)
    line_pred = svg_cls.predict([line_features])[0]
    bar_pred = svg_cls.predict([bar_features])[0]
    scatter_pred = svg_cls.predict([scatter_features])[0]
    heatmap_pred = svg_cls.predict([heatmap_features])[0]
    print(name, line_pred, bar_pred, scatter_pred, heatmap_pred)

/Users/kainylund/opt/anaconda3/envs/arkenv/lib/python3.10/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator StandardScaler from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/kainylund/opt/anaconda3/envs/arkenv/lib/python3.10/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/kainylund/opt/anaconda3/envs/arkenv/lib/python3.10/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator Pipeline from version 1.1.1 when using 

nearest_neighbors 60 60 60 60
linear_SVM 1 1 1 1
RBF_SVM 4 4 4 4
decision_tree 4 4 4 4


/Users/kainylund/opt/anaconda3/envs/arkenv/lib/python3.10/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator StandardScaler from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/kainylund/opt/anaconda3/envs/arkenv/lib/python3.10/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.1 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/kainylund/opt/anaconda3/envs/arkenv/lib/python3.10/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1

random_forest 1 1 1 1
neural_net 60 60 60 60
adaBoost 1 1 1 1
naive_bayes 60 60 60 60


In [30]:
data_dirs = ["./svg_datasets/plotly_export", "./svg_datasets/chartblocks", "./svg_datasets/d3_clean", "./svg_datasets/graphiq_clean"]
#["./svg_datasets/plotly_export", "./svg_datasets/chartblocks", "./svg_datasets/d3_clean", "./svg_datasets/graphiq_clean"]
X = []
y = []
for charts_dir in data_dirs:
    labels_path = f"{charts_dir}/urls.txt"
    id_to_label = {}

    with open(labels_path) as labels_file:
        for label_line in labels_file.readlines():
            label_split = label_line.split(" ")
            sample_id = label_split[0]
            sample_label = label_split[2].split(",")[0]
            if "plotly" in charts_dir:
                id_to_label[sample_id] = chart_type_to_id[sample_label.replace("\n", "")]
            else:
                id_to_label[sample_id] = int(sample_label.replace("\n", ""))


    err_count = 0
    processed_count = 0
    for chart_id, chart_label in tqdm(id_to_label.items()):
        #svg_path = f"{charts_dir}/charts/{chart_id}/svg.txt"
        svg_path = f"{charts_dir}/images/{chart_id}.png"
        #print(svg_path)
        try:
            #with open(svg_path, 'r') as f:
            #    svg_text = f.read()

            #svg_data = BeautifulSoup(svg_text, "xml")
            #svg_data = np.array(PIL.Image.open(svg_path))
            print(svg_path)
            plt.imread(svg_path)
            plt.show()
            plt.gcf().savefig("./test.svg")
            break
            processed_count += 1

        except Exception as err:
            raise (err)
            break
            err_count += 1

    print(charts_dir, processed_count, err_count)

  0%|          | 0/15232 [00:00<?, ?it/s]


./svg_datasets/plotly_export/images/975.png
./svg_datasets/plotly_export 0 0


  0%|          | 0/22557 [00:00<?, ?it/s]

./svg_datasets/chartblocks/images/0d1d7395-9a09-4f97-bc2a-2248dd6aa4ab.png


<Figure size 640x480 with 0 Axes>

...

./svg_datasets/graphiq_clean/images/6621dda0-9002-4c68-8861-f0c0ec061c76.png


<Figure size 640x480 with 0 Axes>

  0%|          | 0/2733 [00:00<?, ?it/s]

./svg_datasets/graphiq_clean 0 0


<Figure size 640x480 with 0 Axes>

In [41]:
plt.imread("./svg_datasets/d3_clean/images/15.png")
plt.show()
plt.gcf().savefig("./test_weird2.svg")

<Figure size 640x480 with 0 Axes>

In [38]:
plt.imread("./test_weird.svg")

UnidentifiedImageError: cannot identify image file './test_weird.svg'

In [48]:
import vtracer

input_path = "./example_imgs/lines.png"
output_path = "./example_imgs/lines.svg"

# Minimal example: use all default values, generate a multicolor SVG
#vtracer.convert_image_to_svg_py(input_path, output_path)

# Single-color example. Good for line art, and much faster than full color:
#vtracer.convert_image_to_svg_py(input_path, output_path, colormode='binary')

# All the bells & whistles
vtracer.convert_image_to_svg_py(input_path,
                                output_path,
                                colormode = 'color',        # ["color"] or "binary"
                                hierarchical = 'stacked',   # ["stacked"] or "cutout"
                                mode = 'spline',            # ["spline"] "polygon", or "none"
                                filter_speckle = 0,         # default: 4
                                color_precision = 6,        # default: 6
                                layer_difference = 16,      # default: 16
                                corner_threshold = 60,      # default: 60
                                length_threshold = 10.0,     # in [3.5, 10] default: 4.0
                                max_iterations = 100,        # default: 10
                                splice_threshold = 45,      # default: 45
                                path_precision = 20          # default: 8
                                )